# Курсовая работа
## Выполнил студент группы БФИ2202 Тимшина Виктория Андреевна

Написать программу, которая будет считывать данные из CSV файла, содержащего
информацию о продажах товаров в магазине. Данные в файле содержатся в следующем
формате:
| Номер заказа | Дата заказа | Название товара | Категория товара | Количество продаж | Цена
за единицу | Общая стоимость |

Необходимо:
1. Рассчитать общую выручку магазина.
2. Найти товар, который был продан наибольшее количество раз.
3. Найти товар, который принес наибольшую выручку.
4. Составить отчет, содержащий информацию об общей выручке магазина, количестве
проданных единиц каждого товар и доле каждого товара в общей выручке.
Для решения задач необходимо использовать структуры данных, такие как массивы и
хеш-таблицы, а также различные алгоритмы обработки данных, например, сортировку и поиск.
Также необходимо учитывать возможные ошибки ввода-вывода и обрабатывать их в
соответствии с требованиями

# Ход работы:

Подключение библиотек:

In [10]:
import pandas as pd
import csv

Класс хэш-таблицы:

In [12]:
# Создаем класс узла списка
class Node:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.next = None

# Создаем класс хэш-таблицы
class HashTable:
    def __init__(self):
        self.capacity = 10 
        self.size = 0 
        self.buckets = [None] * self.capacity 

    # Метод добавления элемента в таблицу
    def add(self, key, value):
        index = hash(key) % self.capacity # хэш-индекс для ключа
        node = self.buckets[index]
        while node: 
            if node.key == key:
                node.value = value
                return
            node = node.next
        new_node = Node(key, value)
        new_node.next = self.buckets[index]
        self.buckets[index] = new_node
        self.size += 1
        if self.size > 0.7 * self.capacity:
            self.resize()

     # Метод получения значения по ключу
    def get(self, key):
        index = hash(key) % self.capacity
        node = self.buckets[index]
        while node:
            if node.key == key: 
                return node.value
            node = node.next
        return None

    # Метод изменения емкости таблицы
    def resize(self):
        new_capacity = self.capacity * 2
        new_buckets = [None] * new_capacity
        # перехешируем все элементы из старой таблицы в новую
        for i in range(self.capacity):
            node = self.buckets[i]
            while node:
                index = hash(node.key) % new_capacity
                if new_buckets[index]:
                    new_node = new_buckets[index]
                    while new_node.next:
                        new_node = new_node.next
                    new_node.next = Node(node.key, node.value)
                else:
                    new_buckets[index] = Node(node.key, node.value)
                node = node.next
        self.capacity = new_capacity
        self.buckets = new_buckets

Функция сортировки выбором:

In [13]:
def selection_sort(arr):      

    for i in range(len(arr)):
        min = i
        
        for j in range(i + 1, len(arr)):
            if arr[j] < arr[min]:
                min = j

        arr[min], arr[i] = arr[i], arr[min]
            
    return arr

Функция для вычисления:
- Общей выручки
- Товара, который был продан наибольшее количество раз
- Товара, который принес наибольшую выручку

In [5]:
def counting(file):
    earnings_table = HashTable() # таблица доходов
    sales_table = HashTable() # таблица продаж
    
    revenue = 0
    count_max_sold_product = 0
    max_sold_product = ""
    max_revenue_product = ""
    revenue_array = []

    next(file)  # пропускаем первую строку с заголовками

    for line in file:
        revenue += int(line[6])

        if count_max_sold_product < int(line[4]):
            count_max_sold_product = int(line[4])
            max_sold_product = line[2]
            
        revenue_array.append(int(line[6]))
        sales_table.add(str(line[6]), line[2])
        earnings_table.add(str(line[2]), line[4])
        earnings_table.add(str(line[2]) + "1", line[6])

    sorted_revenue = selection_sort(revenue_array)

    max_revenue_product = sales_table.get(str(sorted_revenue[-1]))

    return revenue, max_sold_product, max_revenue_product


Выведем результаты:

In [14]:
try:
    with open("products.csv", encoding='utf-8') as file:
        
        csv_file_reader = csv.reader(file, delimiter=";")
        revenue, max_sold_product, max_revenue_product = counting(csv_file_reader)
        
        print(f'Общая выручка магазина: {revenue} рублей')
        print(f'Товар, который был продан наибольшее количество раз: {max_sold_product}')
        print(f'Товар, который принес наибольшую выручку: {max_revenue_product}')

except:
    print('Файл не найден')


Общая выручка магазина: 122625 рублей
Товар, который был продан наибольшее количество раз: Картофель
Товар, который принес наибольшую выручку: Яблоки


Доля выручки каждого товара от общей выручки:

In [15]:
data = pd.read_csv('products.csv', sep=';')

data['Доля от общей выручки в %'] = data['Общая стоимость'] / (revenue / 100)
data['Доля от общей выручки в %'] = data['Доля от общей выручки в %'].round(2).astype(int)

Вывод таблицы:

In [16]:
# Преобразуем таблицу в HTML
html = data[['Название товара', 'Количество продаж', 'Общая стоимость', 'Доля от общей выручки в %']].to_html(index=False)

# Добавляем стили
html = html.replace('<table>', '<table style="background-color: blue; border: 1px solid black;">')
html = html.replace('<th>', '<th style="background-color: lightblue; border: 1px solid black; color: darkblue;">')
html = html.replace('<td>', '<td style="background-color: lightblue; border: 1px solid black; color: darkblue;">')
html = html.replace('</tr>', '<tr style="border: 1px solid black;">')

# Выводим таблицу
from IPython.display import HTML
display(HTML(html))

Название товара,Количество продаж,Общая стоимость,Доля от общей выручки в %
Яблоки,500,25000,20
Апельсины,300,12000,9
Бананы,200,6000,4
Морковь,150,3000,2
Персики,100,6000,4
Огурцы,50,1250,1
Клубника,100,7000,5
Помидоры,75,2625,2
Арбузы,25,2000,1
Картофель,600,7500,6


# Вывод:

Я разработала программу для анализа данных о продажах товаров в магазине из CSV-файла. Посчитала общую выручку, определила товар с наибольшим количеством продаж и товар, который принес наибольшую выручку. Составила отчет с общей выручкой, количеством проданных единиц каждого товара и их долей в общей выручке. 